In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly.express as px 
import yfinance as yf 

# Download the data

In [ ]:
#Download data from Yahoo Finance

##JPM: JP Morgan 
##MS: Morgan Stanley 
##BAC: BofA

tickers = ['JPM', 'MS', 'BAC']
start_date= '2024-01-01'
end_date= '2025-01-01'

#we only need the closed value 
data = yf.download(tickers, start=start_date, end=end_date)['Close']
#data.head()

In [ ]:
#Visualize all of our data 
px.line(data, title='Closing Price for BAC, MS & JPM')
#px.line(data, x=data.index, y=data['MS'], title='Morgan Stanley', color_discrete_sequence=['red'])

# Calculate Returns

## Simple

In [ ]:
#Compute Simple Return : (S(t) - S(t-1))/S(t-1) 

simple_returns = data.pct_change()
simple_returns = simple_returns.dropna()
#simple_returns.head()

## Log

In [ ]:
#Calculate Log Return: ln(S(t) / S(t-1)) 

log_returns = np.log(data/data.shift(1))
log_returns = log_returns.dropna()
#log_returns.head()

# Portfolio Returns

## Simple

In [ ]:
#Portfolio Returns (Rp = w1r1 + w2r2 +w3r3)

weights = np.array([1/3, 1/3, 1/3])
portfolio_simple_returns = simple_returns.dot(weights)
#portfolio_simple_returns.head()

## Log

In [ ]:
#Portfolio log returns 

weights = np.array([1/3, 1/3, 1/3])
portfolio_log_returns = log_returns.dot(weights)
#portfolio_log_returns.head()

## Annualized Portfolio Returns

### Simple

In [ ]:
#Portfolio - Simple Return (annualized)
annualized_simple_return = ((1 + portfolio_simple_returns.mean()) ** 252) - 1
print(annualized_simple_return) 

### Log

In [ ]:
#Portfolio - Log Return (annualized)
annualized_log_return = ((1 + portfolio_log_returns.mean()) ** 252) - 1
print(annualized_log_return)

# Volatility

In [ ]:
#Volatility 
daily_volatility = np.std(portfolio_simple_returns)
annual_volatility = daily_volatility * np.sqrt(252)
print(annual_volatility)

# Alpha & Beta

In [ ]:
#Alpha and beta (SPY Data - S&P 500)
# ^GSPC = S&P 500

benchmark = yf.download('^GSPC', start=start_date, end=end_date)['Close']
benchmark = benchmark.pct_change() #Simple returns for S&P 500
benchmark = benchmark.dropna()
#benchmark.head()


## Beta

In [ ]:
#Beta (= Covariance(Rp, Rm) / Var(Rm))

#Make sure they both have similar nature
portfolio_flat = portfolio_simple_returns.to_numpy().flatten()
benchmark_flat = benchmark.to_numpy().flatten()

cov_matrix = np.cov(portfolio_flat, benchmark_flat)
beta = cov_matrix[0, 1] / cov_matrix[1,1]

if beta < 1: 
    print('Our portfolio is less volatile than the market:')
else: 
        print('Our portfolio is more volatile than the market: ')
print(f'For every 1 percent change in the market, our portfolio tends to change by {beta} in the same direction.')


## Alpha

In [ ]:
#Alpha : excess return of a portfolio relatiove to its benchmark 

risk_free_rate = 0.07
alpha = (np.mean(portfolio_simple_returns) - risk_free_rate/252) - beta * (np.mean(benchmark) - risk_free_rate/252)
alpha = alpha * 252 

print(f'Our portfolio outperformed the benchmark by {100 * alpha} percent.')

# Downside Deviation

In [ ]:
#Downside deviation

negative_returns = portfolio_simple_returns[portfolio_simple_returns < 0]
downside_deviation = np.std(negative_returns)
downside_deviation = downside_deviation * np.sqrt(252)
print(downside_deviation)

# Sharpe Ratio

In [ ]:
#Sharpe Ratio = (Rp - Rf)/ Sigma 

sharpe_ratio = (annualized_simple_return - risk_free_rate) / annual_volatility

if sharpe_ratio > 1: 
    print('This sharpe ratio is considered to be really good:')
    
print(f'For every 1 unit of risk, the portfolio is generating {sharpe_ratio} units of excess return')

# Sortino Ratio

In [ ]:
#Sortino ratio = (Rp - Rf) / sigma(d)
sortino_ratio = (annualized_simple_return - risk_free_rate) / downside_deviation

if sortino_ratio > 1: 
    print('This sortino ratio is considered to be really good:')

print(f'For every unit of downside risk, the portfolio is generating {sortino_ratio} units of excess return.')

# Calmar Ratio & Max. Drawdown

In [ ]:
#Calmar Ratio = Rp / Max Drawdown 
#Max drawdown => Cumulative Return (Worst historical loss from peak to bottom) 

cumulative_simple_returns = (1 + portfolio_simple_returns).cumprod()
max_drawdown = ((cumulative_simple_returns.cummax() - cumulative_simple_returns) / cumulative_simple_returns.cummax()).max()

calmar_ratio = annualized_simple_return / max_drawdown

print(max_drawdown, calmar_ratio)

# Treynor Ratio

(Rp - Rf)/ Beta 

In [ ]:
treynor_ratio = (annualized_simple_return - risk_free_rate) / beta

print(treynor_ratio)

# Value at Risk

In [ ]:
#Value at Risk (historical method) => potential loss in your portfolio 

var_percentage = 95
portfolio_value = 1000000

var = np.percentile(portfolio_simple_returns, 100 - var_percentage) * portfolio_value
print(var)

## Conditional Value at Risk

In [ ]:
#Conditional VaR / Expected Shortfall => E[Loss/Loss > VaR]

c_var = portfolio_simple_returns[portfolio_simple_returns <= np.percentile(portfolio_simple_returns, 100 - var_percentage )].mean()
c_var = c_var * portfolio_value
c_var